In [1]:
!pip install langgraph langsmith langchain langchain_groq langchain_community


[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from typing import Annotated
from typing_extensions import TypedDict

In [10]:
##working with tools
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun

#arxiv and wikipedia 
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [14]:
wiki_tool.invoke("who is actor shahrukh khan")

'Page: Shah Rukh Khan\nSummary: Shah Rukh Khan (pronounced [ˈʃaːɦɾʊx xäːn] ; born 2 November 1965), also known by the initialism SRK, is an Indian actor and film producer who works in Hindi cinema. Referred to in the media as the "Baadshah of Bollywood" and "King Khan", he has appeared in more than 10'

In [12]:
arxiv_tool.invoke("Attention all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with "

In [15]:
tools=[wiki_tool]

In [17]:
##langgraph application
from langgraph.graph.message import add_messages
class State(TypedDict):
    messages:Annotated[list,add_messages]


In [18]:
from langgraph.graph import StateGraph,START,END

In [19]:
graph_builder=StateGraph(State)

In [20]:
from langchain_groq import ChatGroq



In [21]:
from google.colab import userdata
groq_api_key=userdata.get("groq_api_key")

ModuleNotFoundError: No module named 'google.colab'

In [23]:
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

NameError: name 'groq_api_key' is not defined

In [24]:
llm_with_tools=llm.bind_tools(tools=tools)

NameError: name 'llm' is not defined

In [25]:
def chatbot(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

In [27]:
from langgraph.prebuilt import ToolNode,tools_condition


In [30]:
graph_builder.add_node("chatbot",chatbot)
graph_builder.add_edge(START,"chatbot")
tool_node=ToolNode(tools=tools)
graph_builder.add_node("tools",tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)

graph_builder.add_edge("tools","chatbot")
graph_builder.add_edge("chatbot",END)


ValueError: Node `chatbot` already present.

In [32]:
graph=graph_builder.compile()


In [33]:
from IPython.display import Image,display

try:
    dispaly(Image(graph.get_graph().draw_mermaid_png()))
except Exception :
    #This requires some extra dependencies and is optional
    pass 

In [34]:
user_input="Hi there!,My name is krish"
events=graph.stream(
    {"messages":[("user",user_input)]},stream_mode="values"

)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi there!,My name is krish


NameError: name 'llm_with_tools' is not defined

In [35]:
user_input = "what is RLHF."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]},stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what is RLHF.


NameError: name 'llm_with_tools' is not defined